In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
import os
import time

from liblibra_core import *
import libra_py.models.Holstein as Holstein
import util.libutil as comn
from libra_py import data_outs
from libra_py import dynamics_plotting

import qtag_basis
import qtag_calc
import qtag_checks
import qtag_ham
import qtag_init
import qtag_mom
import qtag_prop

ModuleNotFoundError: No module named 'liblibra_core'

In [2]:
class tmp:
    pass

In [3]:
#Parameters for general simulation
dyn_params = {
    "ndof" : 1, "nstates" : 2, "active_state" : 0,
    "dt":0.01, "niter":200, "mass":[1.0],
    "nsnapshots":1, "noutput":10
}

#QTAG-specific parameters, mostly function objects
qtag_params = {
    "init_placement":"grid", "grid_dims" : [25], "a0":[6.0], "rho_cut":1e-12,
    "basis_qtype" : "adpt", "basis_ptype" : "adpt", "basis_atype" : "adpt", "basis_stype" : "frzn",
    "wfq0" : [-2.0], "wfp0" : [0.0], "wfa0" : [1.0], "wfs0" : [0.0],
    "prop_method" : "sync", "decpl_den" : 0.3, "mirror" : "true",
    "mom_calc_type" : "lin_fit", "linfit_beta" : 1e-3},
    "v_approx":'BAT'
}

#Potential parameters for Holstein5
model_params = {
    "model":1,
    "E_n":[0.0,15.8114],
    "x_n":[0.0,1.0],
    "k_n":[10.0,10.0],
    "V":[[0.0,1.0],[1.0,0.0]],
    "alpha":[[0.0,1.5811],[1.5811,0.0]],
    "x_nm":[[0.0,2.0],[2.0,0.0]]
}

In [4]:
#Holstein5 from libra_py/models/Hostein.py
def holstein_mimic(q, params):
    critical_params = ["E_n", "x_n", "k_n" ]
    default_params = { "V": [ [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001],
                              [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001] ],
                       "alpha": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                  [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                       "x_nm": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                 [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                     }
    comn.check_input(params, default_params, critical_params)

    E_n = params["E_n"]
    x_n = params["x_n"]
    k_n = params["k_n"]
    V = params["V"]
    alpha = params["alpha"]
    x_nm = params["x_nm"]
    
    n = len(E_n)

    Hdia = CMATRIX(n,n)
    Sdia = CMATRIX(n,n)
    d1ham_dia = CMATRIXList();  d1ham_dia.append( CMATRIX(n,n) )
    dc1_dia = CMATRIXList();  dc1_dia.append( CMATRIX(n,n) )

    indx = 0

    x = q.col(indx).get(0)

    Sdia.identity()

    for i in range(n):
        Hdia.set(i,i,  (E_n[i] + 0.5*k_n[i]*(x - x_n[i])**2) * (1.0+0.0j) )

    for i in range(n):
        for j in range(n):
            if i!=j:
                Hdia.set(i,j,  V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    for k in [0]:
        #  d Hdia / dR_0
        for i in range(n):
            d1ham_dia[k].set(i,i, (k_n[i] * (x - x_n[i]))*(1.0+0.0j) )

    for k in [0]:
        for i in range(n):
            for j in range(n):
                if i!=j:
                    d1ham_dia[k].set(i,j,  -2.0*alpha[i][j] * (x-x_nm[i][j]) * V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    obj = tmp()
    obj.ham_dia = Hdia
    obj.ovlp_dia = Sdia
    obj.d1ham_dia = d1ham_dia
    obj.dc1_dia = dc1_dia

    return obj

In [5]:
def compute_model(q, model_params, full_id):

    model = model_params["model"]
    res = None
    
    if model==1:
        #res = Holstein.Holstein2(q, model_params, full_id)
        res = holstein_mimic(q, model_params)
    else:
        print("Unrecognized model! Check for correct model import statement.")

    return(res)

In [ ]:
colors = {}
colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   
colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      
colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  
colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

plot_params = {"colors": colors, "clrs_index": clrs_index, "ylim":[-0.02, 0.08], "xlim":[-8, 8]}
dynamics_plotting.plot_surfaces(compute_model, [model_params], [0,1], -7.5, 7.5, 0.05, plot_params)

In [6]:
def run_qtag(dyn_params, qtag_params, compute_model, model_params, prefix):
#Some checks for the user input...
#    comn.check_input(univ,{"ndof":1,"mass":[1836.0],"dt":0.1,"niter":1,"n_data_out":1,"n_snapshots":1},[])
#    comn.check_input(wf0,{},["q","p","a","s"])
#    comn.check_input(traj0,{"rho":1e-10},["placement","grid_dims","a0"])
#    comn.check_input(basis,{"qtype":"adpt","ptype":"adpt","atype":"adpt","stype":"frzn"},[])
#    comn.check_input(mss,{"decpl":0.9},["prop_method"])
#    comn.check_input(mom_params,{"adjust":"unmodified","beta":1e-1},[])
#    comn.check_input(model,{"calc_type":"LHA","coupling":"LHA"},["pot_type","rep"])
#    qtag_init.qtag_checks(univ,wf0,traj0,mss,model,model_params)

#Define function objects to be used in various parts of the calculation...
    initialize,props,vapprox,mom_calc,propagate = qtag_checks.assign_fobj(qtag_params)

#Rename variables locally for convenience...
    ndof = dyn_params['ndof']
    nstates = dyn_params["nstates"]
    active_state = dyn_params["active_state"]
    niter = dyn_params['niter']
    dt = dyn_params['dt']
    mass = dyn_params['mass']

#Initialize the basis parameters {q,p,a,s} and create initial b-matrix...
    try:
        ntraj, qpas=initialize(ndof,nstates,qtag_params)
    except AttributeError:
        sys.exit("Error in params dictionary: 'init_placement' keyword not recognized!")

#Create initial projection vector b...
    bt=qtag_init.coeffs(qtag_params,qpas,active_state)

#Start simulation and walltime variables...
    walltime_start = time.time()
    t=0.0

    for iter in range(niter):
#Calculate system overlap on each state, placed into a super-matrix ovlp...
        ovlp = qtag_calc.super_overlap(ndof,ntraj,nstates,qpas)

#Calculate system Hamiltonian for all states and couplings, placed into a super-matrix hmat...
        hmat = qtag_ham.super_hamiltonian(ndof,ntraj,nstates,mass,qpas,vapprox,compute_model,model_params)
        
#Compute the new coefficient vector c_new...
        ct_new=qtag_ham.basis_diag(ntraj,dt,hmat,ovlp,bt)
    
        etot = qtag_calc.energy(ct_new,hmat)
        pops = qtag_calc.norm(qpas[4],ct_new,ovlp,
                              [state for state in range(nstates)])

        print(etot)
#Update the basis parameters according to the new wavefunction (ct_new)...
        qpasn,bt=propagate(dyn_params,qtag_params,mom_calc,
                           props,model_params,qpas,ct_new,pops)

    print(pops)
    walltime_end = time.time()
    print("Total wall time: ",walltime_end-walltime_start)


In [7]:
run_qtag(dyn_params, qtag_params, compute_model, model_params, "qtag")

19.55555555047286
19.55555554437751
19.555555538065548
19.555555534141952
19.555555530813994
19.55555552846078
19.55555552715985
19.555555526630645
19.55555552516195
19.555555524746175
19.55555552521049
19.555555525714407
19.55555552627383
19.555555527056715
19.555555527457198
19.5555555279791
19.555555528277512
19.55555552871294
19.55555552900805
19.555555529302232
19.55555552959728
19.555555529825423
19.55555553000162
19.55555553015599
19.55555553035748
19.55555553047982
19.555555530503938
19.55555553044842
19.555555530801534
19.555555530404263
19.555555530790432
19.55555553086667
19.55555553121874
19.55555553101457
19.555555531279346
19.555555531080223
19.555555530808064
19.55555553065902
19.55555553046134
19.555555530463007
19.555555530406174
19.55555553057734
19.555555530870567
19.555555531423767
19.555555532511534
19.55555553383086
19.555555535625917
19.555555538047702
19.555555540439375
19.55555554349479
19.555555546643156
19.55555555041527
19.55555555362441
19.5555555565932
19.

()